
### Примерные границы ключевых моментов в процентах:

| Событие                  | Время в минутах | Процент от общей длины (~48 мин) |
|--------------------------|------------------|----------------------------------|
| Cold Open / Teaser       | 0–3              | 0–6%                             |
| Act 1 / Завязка           | 3–12             | 6–25%                            |
| Act 2 / Развитие          | 12–24            | 25–50%                           |
| Midpoint                 | ~24              | 50%                              |
| Act 3 / Подъём к кульминации | 24–36         | 50–75%                           |
| Climax / Кульминация      | 36–42            | 75–87.5%                         |
| Tag / Развязка            | 42–48            | 87.5–100%                        |


In [1]:
import random

def get_act_boundaries(
    total_duration=2700,
    highlight_length=20,
    num_highlights=3,
    allowed_acts=None,
    min_gap=30,
    return_discr=False
):
    """
    Генерирует случайные таймкоды для рекапа на основе классической структуры сериала.
    
    Параметры:
        total_duration (int): общая продолжительность серии в секундах
        highlight_length (int): длина одного клипа в секундах
        num_highlights (int): сколько клипов нужно выбрать
        allowed_acts (list): список индексов актов, из которых брать клипы (например [4, 5])
        min_gap (int): минимальное расстояние между клипами (чтобы не пересекались)
        return_discr=False : возвращает описание границы
    
    Возвращает:
        list of tuples: [(start_sec, end_sec, act_name), ...]
    """
    # Стандартная структура серии
    act_boundaries_percent = [
        ('Cold Open / Teaser', 0.00, 0.06),
        ('Act 1 / Завязка', 0.06, 0.25),
        ('Act 2 / Развитие', 0.25, 0.50),
        ('Midpoint', 0.50, 0.50),
        ('Act 3 / Подъём к кульминации', 0.50, 0.75),
        ('Climax / Кульминация', 0.75, 0.875),
        ('Tag / Развязка', 0.875, 1.00)
    ]

    # Переводим в секунды
    act_boundaries_seconds = []
    for name, start_p, end_p in act_boundaries_percent:
        start_s = int(total_duration * start_p)
        end_s = int(total_duration * end_p)
        act_boundaries_seconds.append((name, start_s, end_s))

    if allowed_acts is None:
        allowed_acts = list(range(len(act_boundaries_seconds)))

    highlights = []

    def random_clip(start, end, length):
        """Возвращает случайный клип заданной длины внутри диапазона"""
        available_start = max(start, end - length - 1)
        if available_start >= end - 1:
            return None  # невозможно выбрать клип такой длины
        clip_start = random.randint(available_start, end - 1)
        return (clip_start, clip_start + length)

    attempts = 0
    max_attempts = 20

    while len(highlights) < num_highlights and attempts < max_attempts:
        # Выбираем случайный акт
        act_idx = random.choice(allowed_acts)
        act_name, a_start, a_end = act_boundaries_seconds[act_idx]

        # Пробуем создать клип
        clip = random_clip(a_start, a_end, highlight_length)
        if clip is None:
            attempts += 1
            continue

        # Проверяем, чтобы не пересекался с предыдущими
        if return_discr:
            if all(abs(clip[0] - h[0]) > min_gap for h in highlights):
                highlights.append((*clip, act_name))
                attempts = 0
            else:
                attempts += 1
        else:
            if all(abs(clip[0] - h[0]) > min_gap for h in highlights):
                highlights.append(clip)
                attempts = 0
            else:
                attempts += 1

    return highlights

In [2]:
get_act_boundaries()

[(658, 678), (2017, 2037), (2357, 2377)]

In [13]:
get_act_boundaries(return_discr=True)

[(144, 164, 'Cold Open / Teaser'),
 (658, 678, 'Act 1 / Завязка'),
 (1329, 1349, 'Act 2 / Развитие')]